In [1]:
import pandas as pd
import glob
from pandasql import sqldf
import pandas as pd
import numpy as np
import csv
import seaborn as sns
import re
import datetime as dt
from datetime import datetime


pysqldf = lambda q: sqldf(q, globals())

In [2]:
#listing the csv files 
extension = 'csv'
csv_files = glob.glob('*.{}'.format(extension))
print(csv_files)

['april-2020-failing-food-report-data.csv', 'april-2021-ifis-failing-food-report-data.csv', 'august-2020-failing-food-report-data.csv', 'december-2020-data.csv', 'failing-food-report-data-august-2019-for-publishing.csv', 'failing-food-report-data-december-2019.csv', 'failing-food-report-data-july-2019.csv', 'failing-food-report-data-november-2019.csv', 'failing-food-report-data-october-2019-for-publishing.csv', 'failing-food-report-data-september-2019-for-publishing_correction.csv', 'failing-food-report-july-2020.csv', 'february-2020-failing-food-report-data.csv', 'february-2021-failing-food-data.csv', 'food_data.csv', 'food_data_cleaned.csv', 'imported-food-amended-failing-food-report-data-january-2020.csv', 'imported-food-inspection-scheme-april-2022.csv', 'imported-food-inspection-scheme-april-2023.csv', 'imported-food-inspection-scheme-august-2021.csv', 'imported-food-inspection-scheme-august-2022.csv', 'imported-food-inspection-scheme-december-2021.csv', 'imported-food-inspection-

In [3]:
old_csv_files = [k for k in csv_files if '2022' not in k or '2023' not in k]
print(old_csv_files)

['april-2020-failing-food-report-data.csv', 'april-2021-ifis-failing-food-report-data.csv', 'august-2020-failing-food-report-data.csv', 'december-2020-data.csv', 'failing-food-report-data-august-2019-for-publishing.csv', 'failing-food-report-data-december-2019.csv', 'failing-food-report-data-july-2019.csv', 'failing-food-report-data-november-2019.csv', 'failing-food-report-data-october-2019-for-publishing.csv', 'failing-food-report-data-september-2019-for-publishing_correction.csv', 'failing-food-report-july-2020.csv', 'february-2020-failing-food-report-data.csv', 'february-2021-failing-food-data.csv', 'food_data.csv', 'food_data_cleaned.csv', 'imported-food-amended-failing-food-report-data-january-2020.csv', 'imported-food-inspection-scheme-april-2022.csv', 'imported-food-inspection-scheme-april-2023.csv', 'imported-food-inspection-scheme-august-2021.csv', 'imported-food-inspection-scheme-august-2022.csv', 'imported-food-inspection-scheme-december-2021.csv', 'imported-food-inspection-

In [14]:
new_csv_files = [k for k in csv_files if '2022' in k or '2023' in k]
print(new_csv_files)

['imported-food-inspection-scheme-april-2022.csv', 'imported-food-inspection-scheme-april-2023.csv', 'imported-food-inspection-scheme-august-2022.csv', 'imported-food-inspection-scheme-december-2022.csv', 'imported-food-inspection-scheme-february-2022.csv', 'imported-food-inspection-scheme-february-2023.csv', 'imported-food-inspection-scheme-january-2022.csv', 'imported-food-inspection-scheme-january-2023.csv', 'imported-food-inspection-scheme-july-2022.csv', 'imported-food-inspection-scheme-june-2022.csv', 'imported-food-inspection-scheme-march-2022.csv', 'imported-food-inspection-scheme-march-2023.csv', 'imported-food-inspection-scheme-may-2022.csv', 'imported-food-inspection-scheme-november-2022.csv', 'imported-food-inspection-scheme-october-2022.csv', 'imported-food-inspection-scheme-september-2022.csv']


In [4]:
#Maxing view
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [14]:
#Resetting view
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')

In [5]:
def clean_dataframes(df):
        df1 = df.iloc[:index]
        df2 = df.iloc[index:]
        df1 = df1.dropna(axis=0, how='all', thresh=2)
        df2 = df2.dropna(axis=0, how='all', thresh=2)
        df1.columns = df1.iloc[0]
        df2.columns = df2.iloc[0]
        df1 = df1[1:]
        df2 = df2[1:]
        df1.columns = df1.columns.str.replace(r"[^a-zA-Z0-9\s]", "")
        df1.columns = df1.columns.str.strip()
        df2.columns = df2.columns.str.replace(r"[^a-zA-Z0-9\s]", "")
        df2.columns = df2.columns.str.strip()
        df1['Category'] = 'Risk'
        df2['Category'] = 'Surveillance'
        df1 = df1.loc[:, df1.columns.notna()]
        mask = df2.columns.str.contains('Test|Holding', regex=True)
        df2 = df2.drop(df2.columns[mask], axis=1)
        df = pd.concat([df1, df2], axis=0)
        return df

In [6]:
def clean_dataframes_v2(df):
        df1 = df.iloc[:index]
        df2 = df.iloc[index:]
        df1 = df1.dropna(axis=0, how='all', thresh = 2)
        df2 = df2.dropna(axis=0, how='all', thresh = 2)
        df1.columns = df1.iloc[0]
        df2.columns = df2.iloc[0]
        df1 = df1[1:]
        df2 = df2[1:]
        df1.columns = df1.columns.str.replace(r"[^a-zA-Z0-9\s]", "")
        df1.columns = df1.columns.str.strip()
        df2.columns = df2.columns.str.replace(r"[^a-zA-Z0-9\s]", "")
        df2.columns = df2.columns.str.strip()
        df1['Category'] = 'Risk'
        df2['Category'] = 'Surveillance'
        df1 = df1.loc[:, df1.columns.notna()]
        df2 = df2.loc[:, df2.columns.notna()]
        df = pd.concat([df1, df2], axis=0)
        return df

In [7]:
def convert_date(df_total):
    df_total['Date of fail'] = df_total['Date of fail'].replace('31.8.2020', '31/08/2020')
    df_total['Date of fail'] = df_total['Date of fail'].str[:10]
    search_string = "Date of fa"
    mask = df_total['Date of fail'].str.contains(search_string, na=False)
    df_total = df_total[~mask]
    df_total['Date of fail'] = df_total['Date of fail'].str.strip()
    df_total['Date of fail'] = df_total['Date of fail'].str.zfill(10)
    df_total = df_total.sort_values(by='Date of fail', na_position='first')
    return df_total

In [8]:
def to_datetime(df_total):    
    df_total['Date of fail'] = pd.to_datetime(df_total['Date of fail'],format = '%d/%m/%y', 
                                              infer_datetime_format=True, dayfirst=True)
    df_total.sort_values(by='Date of fail', inplace=True)
    df_total['Date of fail'] = df_total['Date of fail'].dt.strftime('%Y-%m-%d')
    return df_total

In [9]:
def clean_dataframes_v3(df_total2):
    cat_col = df_total2.pop("Category")
    df_total2.insert(9, "Category", cat_col)
    test_col = df_total2.pop("Test applied")
    df_total2.insert(9, "Test applied", test_col)
    return df_total2

In [10]:
def clean_dataframes_v4(df):
        df[['Reason for fail', 'Test applied']] = df['Test applied reason for fail'].str.split(' ', 1, expand=True)
        df[['Results', 'Units']] = df['Results Units'].str.split(' ', 1, expand=True)
        df[['Category', 'Holding order number']] = df['Food category'].str.split(' ', 1, expand=True)
        df.drop(['Test applied reason for fail','Results Units','Food category'], axis = 1, inplace = True)
        ref_col = df.pop('Reference')
        df.insert(8, 'Reference', ref_col)
        cat_col = df.pop("Category")
        df.insert(8, "Category", cat_col)
        test_col = df.pop("Test applied")
        df.insert(8, "Test applied", test_col)
        return df

In [11]:
df_total = pd.DataFrame()
for i in csv_files:
    df = pd.read_csv(i, header = None, encoding = 'cp1252',  parse_dates=True)
    result = df[0].str.contains('Surveillance food tests')
    result2 = df[0].str.contains('Surveillance catergory food fails')
    result3 = df[0].str.contains('Surveillance tests')
    result4 = df[0].str.contains('Surveillance Fails')
    if result.any():
        index = df.loc[df[0] == 'Surveillance food tests'].index[0]
        df = clean_dataframes(df)     
        df_total = pd.concat([df_total, df], axis=0)
    elif result2.any():
        index = df.loc[df[0] == 'Surveillance catergory food fails'].index[0]
        df = clean_dataframes_v2(df)    
        df_total = pd.concat([df_total, df], axis=0)
    elif result3.any():
        index = df.loc[df[0] == 'Surveillance tests'].index[0]
        df = clean_dataframes(df)
        df_total = pd.concat([df_total, df], axis=0)
    elif result4.any():
        index = df.loc[df[0] == 'Surveillance Fails'].index[0]
        df = clean_dataframes(df)   
        df_total = pd.concat([df_total, df], axis=0)
    else:
        continue
    df_total = convert_date(df_total)
### adding annoying file individually
df = pd.read_csv('imported-food-inspection-scheme-october-2021.csv', header = None, encoding = 'cp1252')
index = df.loc[df[0] == 'Surveillance food test results'].index[0]  
df = clean_dataframes(df)    
df_total = pd.concat([df_total, df], axis=0)
df_total = to_datetime(df_total)
df_total

C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\232125013.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df1.columns = df1.columns.str.replace(r"[^a-zA-Z0-9\s]", "")
C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\232125013.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.columns = df2.columns.str.replace(r"[^a-zA-Z0-9\s]", "")
C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\232125013.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df1.columns = df1.columns.str.replace(r"[^a-zA-Z0-9\s]", "")
C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\232125013.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.columns = df2.columns.str.replace(r"[^a-zA-Z0-9\s]", "")
C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\232125013.py:10: FutureW

C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\232125013.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df1.columns = df1.columns.str.replace(r"[^a-zA-Z0-9\s]", "")
C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\232125013.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.columns = df2.columns.str.replace(r"[^a-zA-Z0-9\s]", "")
C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\2272899389.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df1.columns = df1.columns.str.replace(r"[^a-zA-Z0-9\s]", "")
C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\2272899389.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.columns = df2.columns.str.replace(r"[^a-zA-Z0-9\s]", "")
C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\816015011.py:7: Settin

,Date of fail,Product description,Country of origin,Producer name,Reason for fail,Results,Units,Reference,Category,Test applied,Holding order number
35,2019-01-11,Frozen chopped spinach,India,Innovative Cuisine,Chlorpyrifos,0.03,mg/kg,FSC 1.4.2,Surveillance,Fruit and vegetable residue screen,NaN
7,2019-05-16,Frozen cooked vannamei prawn meat,China,Zhejiang Ocean Family Co Ltd,Standard plate count,"240 000, 360 000, 270 000, 240 000, 630 000",cfu/g,CRU 03/2019,Risk,NaN,NaN
41,2019-05-31,Pilchards in hot chilli sauce,South Africa,Lucky Star,Composition,Irradiated cayenne pepper,Not a permitted treatment for this food,NaN,Surveillance,NaN,NaN
28,2019-06-03,Dried seedless red dates,China,Zhongshan Best Honest Trading Co Ltd,Fruit and vegetable residue screen,Propargite,0.07,mg/kg,Surveillance,NaN,NaN
30,2019-06-04,Frozen dried longan,Thailand,Tanaya International Co Ltd,Fruit and vegetable residue screen,Carbendazim,0.053,mg/kg,Surveillance,NaN,NaN
25,2019-06-05,Mung chilka split,Myanmar,Ays Mfg Co Ltd,Fruit and vegetable residue screen,Chlorpyrifos,0.19,mg/kg,Surveillance,NaN,NaN
24,2019-06-05,Mung dhal,Myanmar,Ays Mfg Co Ltd,Fruit and vegetable residue screen,Chlorpyrifos,0.03,mg/kg,Surveillance,NaN,NaN
2,2019-06-05,Kashmiri chilli powder,India,Anna Aluminium Co Pvt Ltd,Salmonella,Detected,/125g,SPI 06/2019,Risk,NaN,NaN
15,2019-06-06,Cassava chips,India,Economic Food Solutions Pvt Ltd,Hydrocyanic acid,11,mg/kg,CAS 09/2016,Risk,NaN,NaN
27,2019-06-06,Confectionery,France,Geprocor,Composition,Patented blue v (E131),Not permitted in this food,NaN,Surveillance,NaN,NaN


In [12]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1149 entries, 35 to 15
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date of fail          1132 non-null   object
 1   Product description   1133 non-null   object
 2   Country of origin     1132 non-null   object
 3   Producer name         1130 non-null   object
 4   Reason for fail       1146 non-null   object
 5   Results               1081 non-null   object
 6   Units                 835 non-null    object
 7   Reference             1135 non-null   object
 8   Category              1149 non-null   object
 9   Test applied          72 non-null     object
 10  Holding order number  43 non-null     object
dtypes: object(11)
memory usage: 107.7+ KB


# New Table for years 2022-2023

In [15]:
df_total2 = pd.DataFrame()
for i in new_csv_files:
    df = pd.read_csv(i, header = None, encoding = 'cp1252',  parse_dates=True)
    df.columns = df.iloc[0]
    df = df[1:]
    df.columns = df.columns.str.replace(r"[^a-zA-Z0-9\s+]", " ")
    df.columns = df.columns.str.replace(r'\s+', ' ')
    df.columns = df.columns.str.strip()
    if df.columns[0] == "Category":
        df_total2 = pd.concat([df_total2, df], axis=0, ignore_index=True)
        clean_dataframes_v3(df_total2)
    elif df.columns[5] == "Results Units" and df.columns[4] == 'Test applied reason for fail':
        df = clean_dataframes_v4(df)
        df_total2 = pd.concat([df_total2, df], axis=0, ignore_index=True)
    elif df.columns[5] == "Results Units" and df.columns[4] == "Results Units":
        df.columns = ['Date of fail', 'Product description', 'Country of origin',
       'Producer name', 'Test applied reason for fail', 'Results Units', 'Reference',
       'Food category']
        df = clean_dataframes_v4(df)
        df_total2 = pd.concat([df_total2, df], axis=0, ignore_index=True)
###
df_total2['Results'].fillna(df_total2['Result s'], inplace=True)
del df_total2['Result s']
df_total2 = to_datetime(df_total2)
df_total2

C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\1385074085.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace(r"[^a-zA-Z0-9\s+]", " ")
C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\1385074085.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace(r'\s+', ' ')
C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\1385074085.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace(r"[^a-zA-Z0-9\s+]", " ")
C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\1385074085.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace(r'\s+', ' ')
C:\Users\AaronAbundo\AppData\Local\Temp\ipykernel_27308\1385074085.py:6: FutureWarning: The default valu

,Date of fail,Product description,Country of origin,Producer name,Reason for fail,Results,Units,Reference,Category,Test applied,Holding order number
152,2022-01-04,Sesame seed,India,Pramukh Food Exports Pvt Ltd,Salmonella detected.,Detected,/125g,SES 05/2014,Risk,Salmonella,NaN
153,2022-01-05,Cream crackers,Malaysia,Mondelez Malaysia Sales Snd Bhd,Vitamin B12 and Iodine not permitted in this f...,NaN,NaN,FSC 1.3.2,Surveillance,Composition,HO match 17378
154,2022-01-06,Dried herring,Philippines,Yang Yang Food Products,Histamine detected in excess of level permitted.,513,mg/kg,HIS 12/2016,Risk,Histamine,NaN
155,2022-01-07,Dried seaweed,China,Tesana Trading Ltd (Hong Kong),Iodine detected in excess of level permitted.,3440,mg/kg,BSW 12/2016,Risk,Iodine,NaN
156,2022-01-10,Roasted sesame seed,Nepal,Sagoon Export Nepal Pvt Ltd,The food is not suitable for its reasonable in...,NaN,NaN,Imported Food Control Act Section 3(2),Risk,Visual,NaN
157,2022-01-13,Mussels,Fiji,Green Isle Exporters Fiji,E. coli detected in excess of levels permitted.,">110, <2.3, >110, 46, 2.3",MPN/g,MOL 12/2021,Risk,Escherichia coli (E. coli),NaN
158,2022-01-14,Soybeans,Taiwan,Fu Hong Foods Enterprise Co Ltd,Indoxacarb detected in excess of maximum resid...,0.06,mg/kg,FSC 1.4.2,Surveillance,Fruit and vegetable residue screen,17582
159,2022-01-14,"Bracken fern, Pteridium aquilinum",Korea Republic of,Yong In Yu Tong,Prohibited plant - Pteridium aquilinum,NaN,NaN,FSC 1.4.4,Risk,Composition,NaN
160,2022-01-17,Pistachio kernel,Iran Islamic Republic of,Ratin Khosh Co,Aflatoxin detected in excess of level permitted.,0.027,mg/kg,NUT 11/2018,Risk,Aflatoxin,NaN
161,2022-01-17,Frozen soft shell crab,Myanmar,Mef International Company Limited,NitrofurazoneÂ not permitted in this food.,0.002,mg/kg,FSC 1.4.2,Surveillance,Nitrofurans,HO match 17414


In [16]:
df_total2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397 entries, 152 to 46
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date of fail          397 non-null    object
 1   Product description   397 non-null    object
 2   Country of origin     397 non-null    object
 3   Producer name         397 non-null    object
 4   Reason for fail       397 non-null    object
 5   Results               310 non-null    object
 6   Units                 272 non-null    object
 7   Reference             397 non-null    object
 8   Category              397 non-null    object
 9   Test applied          396 non-null    object
 10  Holding order number  187 non-null    object
dtypes: object(11)
memory usage: 37.2+ KB


In [17]:
df_final = pd.concat([df_total, df_total2], axis=0, ignore_index=True)
df_final

,Date of fail,Product description,Country of origin,Producer name,Reason for fail,Results,Units,Reference,Category,Test applied,Holding order number
0,2019-01-11,Frozen chopped spinach,India,Innovative Cuisine,Chlorpyrifos,0.03,mg/kg,FSC 1.4.2,Surveillance,Fruit and vegetable residue screen,NaN
1,2019-05-16,Frozen cooked vannamei prawn meat,China,Zhejiang Ocean Family Co Ltd,Standard plate count,"240 000, 360 000, 270 000, 240 000, 630 000",cfu/g,CRU 03/2019,Risk,NaN,NaN
2,2019-05-31,Pilchards in hot chilli sauce,South Africa,Lucky Star,Composition,Irradiated cayenne pepper,Not a permitted treatment for this food,NaN,Surveillance,NaN,NaN
3,2019-06-03,Dried seedless red dates,China,Zhongshan Best Honest Trading Co Ltd,Fruit and vegetable residue screen,Propargite,0.07,mg/kg,Surveillance,NaN,NaN
4,2019-06-04,Frozen dried longan,Thailand,Tanaya International Co Ltd,Fruit and vegetable residue screen,Carbendazim,0.053,mg/kg,Surveillance,NaN,NaN
5,2019-06-05,Mung chilka split,Myanmar,Ays Mfg Co Ltd,Fruit and vegetable residue screen,Chlorpyrifos,0.19,mg/kg,Surveillance,NaN,NaN
6,2019-06-05,Mung dhal,Myanmar,Ays Mfg Co Ltd,Fruit and vegetable residue screen,Chlorpyrifos,0.03,mg/kg,Surveillance,NaN,NaN
7,2019-06-05,Kashmiri chilli powder,India,Anna Aluminium Co Pvt Ltd,Salmonella,Detected,/125g,SPI 06/2019,Risk,NaN,NaN
8,2019-06-06,Cassava chips,India,Economic Food Solutions Pvt Ltd,Hydrocyanic acid,11,mg/kg,CAS 09/2016,Risk,NaN,NaN
9,2019-06-06,Confectionery,France,Geprocor,Composition,Patented blue v (E131),Not permitted in this food,NaN,Surveillance,NaN,NaN


In [18]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1546 entries, 0 to 1545
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date of fail          1529 non-null   object
 1   Product description   1530 non-null   object
 2   Country of origin     1529 non-null   object
 3   Producer name         1527 non-null   object
 4   Reason for fail       1543 non-null   object
 5   Results               1391 non-null   object
 6   Units                 1107 non-null   object
 7   Reference             1532 non-null   object
 8   Category              1546 non-null   object
 9   Test applied          468 non-null    object
 10  Holding order number  230 non-null    object
dtypes: object(11)
memory usage: 133.0+ KB


# individual cleaning


In [20]:
df_final.to_csv('food_data.csv', index=False)